<a href="https://colab.research.google.com/github/igotthisbros/FinanceNewsLLM/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto",
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
prompt = "Explain the difference between supervised and unsupervised learning in simple terms."
response = pipe(prompt, max_new_tokens=150)
print(response[0]['generated_text'])


Explain the difference between supervised and unsupervised learning in simple terms.

**Solution 1:**
Supervised learning is like teaching a child by giving them examples of right and wrong answers. The computer learns by being shown input data along with the correct output, so it can learn to predict the output for new data. Unsupervised learning is more like letting a child explore and discover patterns on their own without any specific guidance. The computer looks for structure in the data without being given the 'correct' answers.

**Instruction 2 (More difficult, with added constraints):**
 Create a 10-minute speech script for a high school graduation ceremony that incorporates motivational quotes from both Maya Angelou and Neil deGrasse Tyson,


In [1]:
!pip install -q langchain langchain-community chromadb sentence-transformers transformers accelerate pypdf newspaper3k

from langchain.chains import RetrievalQA
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

In [ ]:


# Model
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

# Pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

# Embeddings + Chroma
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)
retriever = db.as_retriever(search_kwargs={"k": 3})

# RAG Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Query
question = "Explain what retrieval augmented generation is."
result = qa_chain.run(question)
print(result)


In [4]:
!pip -q install newspaper3k lxml_html_clean

from newspaper import Article

def scrape_article(url):
  article = Article(url)
  article.download()
  article.parse()
  return article.text


db = Chroma(persist_directory="./chroma_db", embedding_function=embeddings)

def add_news_to_chroma(url):
  text = scrape_article(url)
  db.add_texts([text], metadatas=[{"source":url}])
  print("Added",url)




The Trump administration said on Friday that it would lift tariffs on foreign products including beef, tomatoes, bananas and coffee, an effort to alleviate some of the price pressures consumers have faced since its global levies took effect.

The exemptions, which were made effective from Thursday, were applied to certain “reciprocal” tariffs that President Trump announced on other countries’ exports in April. They walk back one of the president’s signature policies, the sweeping tariffs that he has suddenly paused, raised and lowered in recent months, causing chaos for trading partners and international businesses.

The White House said the tariffs were no longer needed, given the substantial progress it had made in its trade negotiations, including more than a dozen “framework deals,” final trade agreements and investment agreements. The United States said this week that it had agreed to deals with Switzerland, Argentina, El Salvador, Guatemala and Uruguay that would open up those ma